In [1]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [2]:
documents

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.'),
 Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.')]

In [13]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_groq.chat_models import ChatGroq

groq_api_key = os.getenv("GROQ_API_KEY")

os.environ['HF_TOKEN'] = os.getenv("HF_TOKEN")

llm = ChatGroq(api_key=groq_api_key, model="Llama3-8b-8192")

In [4]:
from langchain_huggingface import HuggingFaceEmbeddings

embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

g:\Study\Gen AI\Langchain\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Ignored error while writing commit hash to C:\Users\Swastik\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2\refs\main: [Errno 13] Permission denied: 'C:\\Users\\Swastik\\.cache\\huggingface\\hub\\models--sentence-transformers--all-MiniLM-L6-v2\\refs\\main'.


In [5]:
#vector stores
from langchain_chroma import Chroma

vectorstore = Chroma.from_documents(documents=documents, embedding=embedding)

In [6]:
vectorstore.similarity_search("cat")

[Document(id='f1115f8a-caab-4c43-984d-934efba48729', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='c672a0a3-fc56-4b42-85e1-326bde68fa07', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(id='7d169f41-4bac-4e77-8d8b-42e607c439d2', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
 Document(id='18933c76-b30b-45e2-998b-61527272106c', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.')]

In [7]:
# Async query
await vectorstore.asimilarity_search("cat")

[Document(id='f1115f8a-caab-4c43-984d-934efba48729', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='c672a0a3-fc56-4b42-85e1-326bde68fa07', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(id='7d169f41-4bac-4e77-8d8b-42e607c439d2', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
 Document(id='18933c76-b30b-45e2-998b-61527272106c', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.')]

In [9]:
vectorstore.similarity_search_with_score("Cat")

[(Document(id='f1115f8a-caab-4c43-984d-934efba48729', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
  0.9351057410240173),
 (Document(id='c672a0a3-fc56-4b42-85e1-326bde68fa07', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
  1.574089765548706),
 (Document(id='7d169f41-4bac-4e77-8d8b-42e607c439d2', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
  1.5956902503967285),
 (Document(id='18933c76-b30b-45e2-998b-61527272106c', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
  1.665792465209961)]

### Retrievers

In [10]:
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1) #Lambda will be mapped to all the values in batch anf bind will return the top results
retriever.batch(['cat','dog'])

[[Document(id='f1115f8a-caab-4c43-984d-934efba48729', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(id='c672a0a3-fc56-4b42-85e1-326bde68fa07', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]]

In [11]:
# vectorstore as_retriever
retriever = vectorstore.as_retriever(
    search_type='similarity',
    search_kwargs={'k':1}
)
retriever.batch(['cat','dog'])

[[Document(id='f1115f8a-caab-4c43-984d-934efba48729', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(id='c672a0a3-fc56-4b42-85e1-326bde68fa07', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]]

In [14]:
# RAG
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Answer this question using the provided context only.
{question}

Context:
{context}
"""
prompt = ChatPromptTemplate.from_messages([("human", message)])

rag_chain = {'context':retriever, 'question':RunnablePassthrough()}|prompt|llm

In [16]:
response = rag_chain.invoke("tell me about dogs?")
response.content

'According to the provided context, dogs are great companions, known for their loyalty and friendliness.'